# Анализ информации

Импортируем информацию из `result.json`.

In [1]:
import json

DATA_PATH = "data/result.json"

with open(DATA_PATH) as f:
    data = json.load(f)

In [2]:
data.keys()

dict_keys(['about', 'personal_information', 'profile_pictures', 'contacts', 'frequent_contacts', 'other_data', 'chats', 'left_chats'])

## Чаты

Рассмотрим сущность `chat`.

In [3]:
chats = data['chats']['list']
print(len(chats))
for i in range(len(chats)):
    print(f"{i} : {chats[i].keys()}")

284
0 : dict_keys(['name', 'type', 'id', 'messages'])
1 : dict_keys(['name', 'type', 'id', 'messages'])
2 : dict_keys(['name', 'type', 'id', 'messages'])
3 : dict_keys(['name', 'type', 'id', 'messages'])
4 : dict_keys(['name', 'type', 'id', 'messages'])
5 : dict_keys(['name', 'type', 'id', 'messages'])
6 : dict_keys(['name', 'type', 'id', 'messages'])
7 : dict_keys(['name', 'type', 'id', 'messages'])
8 : dict_keys(['name', 'type', 'id', 'messages'])
9 : dict_keys(['name', 'type', 'id', 'messages'])
10 : dict_keys(['name', 'type', 'id', 'messages'])
11 : dict_keys(['type', 'id', 'messages'])
12 : dict_keys(['name', 'type', 'id', 'messages'])
13 : dict_keys(['name', 'type', 'id', 'messages'])
14 : dict_keys(['name', 'type', 'id', 'messages'])
15 : dict_keys(['name', 'type', 'id', 'messages'])
16 : dict_keys(['name', 'type', 'id', 'messages'])
17 : dict_keys(['name', 'type', 'id', 'messages'])
18 : dict_keys(['name', 'type', 'id', 'messages'])
19 : dict_keys(['name', 'type', 'id', 'messag

`chat` имеет поля `name`, `type`, `id`, `messages`.

**NOTA BENE**:

Тип `saved_messages` не имеет поля `name`. Необходимо это учитывать при парсинге!

In [4]:
chats[11]['type']

'saved_messages'

In [5]:
chats[11].keys()

dict_keys(['type', 'id', 'messages'])

Рассмотрим типы чатов.

In [6]:
sorted(list(set(map(lambda chat: chat['type'], chats))))

['bot_chat',
 'personal_chat',
 'private_group',
 'private_supergroup',
 'saved_messages']

`chat.type` может принимать значения `bot_chat`, `personal_chat`, `private_group`, `private_supergroup`, `saved_messages`.

## Сообщения

Рассмотрим сущность `messages`.

In [7]:
def extract_all_messages(chats):
    for chat in chats:
        for message in chat['messages']:
            yield message

In [8]:
message_keys = sorted(list({str(list(message.keys())) for message in extract_all_messages(chats)}))
print(len(message_keys))
message_keys

213


["['id', 'type', 'date', 'date_unixtime', 'actor', 'actor_id', 'action', 'boosts', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'actor', 'actor_id', 'action', 'discard_reason', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'actor', 'actor_id', 'action', 'duration', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'actor', 'actor_id', 'action', 'duration_seconds', 'discard_reason', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'actor', 'actor_id', 'action', 'inviter', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'actor', 'actor_id', 'action', 'members', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'actor', 'actor_id', 'action', 'message_id', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'actor', 'actor_id', 'action', 'new_title', 'new_icon_emoji_id', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'actor', '

`message` имеет довольно много атрибутов. При парсинге всех сообщений из *моих* (см. прим.) чатов получилось 213 различных комбинаций атрибутов.


*Прим.:* не исключено, что в моих чатах нет каких-то сообщений, которые поддерживает тг, но просто никто не писал их лично мне.

In [9]:
sorted(list({str(message['type']) for message in extract_all_messages(chats)}))

['message', 'service']

In [10]:
sorted(list({message['action'] for message in extract_all_messages(chats) if message['type'] == 'service'}))

['boost_apply',
 'create_channel',
 'create_group',
 'delete_group_photo',
 'edit_group_photo',
 'edit_group_title',
 'group_call',
 'group_call_scheduled',
 'invite_members',
 'invite_to_group_call',
 'join_group_by_link',
 'join_group_by_request',
 'joined_telegram',
 'migrate_from_group',
 'migrate_to_supergroup',
 'phone_call',
 'pin_message',
 'remove_members',
 'topic_created',
 'topic_edit']

Сообщения имеют тип (поле `type`), принимающий 2 значения: `'message'` и `'service'`. Сообщения типа `service` - системные сообщения самого telegram. Примеры этих сообщений см. выше. Скорее всего при парсинге и обработке мы просто будем их игнорировать.

In [13]:
def message_filter(message):
    return message['type'] != 'service'
message_keys = sorted(list({str(list(message.keys())) for message in extract_all_messages(chats) if message_filter(message)}))
print(len(message_keys))
message_keys

200


["['id', 'type', 'date', 'date_unixtime', 'edited', 'edited_unixtime', 'from', 'from_id', 'author', 'photo', 'width', 'height', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'edited', 'edited_unixtime', 'from', 'from_id', 'author', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'edited', 'edited_unixtime', 'from', 'from_id', 'file', 'media_type', 'mime_type', 'duration_seconds', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'edited', 'edited_unixtime', 'from', 'from_id', 'file', 'media_type', 'mime_type', 'duration_seconds', 'width', 'height', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'edited', 'edited_unixtime', 'from', 'from_id', 'file', 'media_type', 'performer', 'title', 'mime_type', 'duration_seconds', 'text', 'text_entities']",
 "['id', 'type', 'date', 'date_unixtime', 'edited', 'edited_unixtime', 'from', 'from_id', 'file', 'mime_type', 'text', 'text_entities']",
 "['id', 'type', 'date'

Игнорируя сообщения типа `service`, у нас получилось 200 различных комбинаций атрибутов (на 13 меньше, чем раньше).

**Дальше при исследовании мы будем игнорировать сообщения типа `service`!**

Рассмотрим внимательнее отдельные параметры сообщений.

In [19]:
def message_filter(message):
    return message['type'] != 'service'
message_keys = sorted(list({tuple(message.keys()) for message in extract_all_messages(chats) if message_filter(message)}))
params = sorted(list({par for keys in message_keys for par in keys }))
print(len(params))
params

33


['author',
 'contact_information',
 'contact_vcard',
 'date',
 'date_unixtime',
 'duration_seconds',
 'edited',
 'edited_unixtime',
 'file',
 'forwarded_from',
 'from',
 'from_id',
 'height',
 'id',
 'inline_bot_buttons',
 'live_location_period_seconds',
 'location_information',
 'media_type',
 'mime_type',
 'performer',
 'photo',
 'poll',
 'reply_to_message_id',
 'reply_to_peer_id',
 'saved_from',
 'sticker_emoji',
 'text',
 'text_entities',
 'thumbnail',
 'title',
 'type',
 'via_bot',
 'width']

In [28]:
required_params = []
optional_params = []
for par in params:
    if all(map(lambda keys: par in keys, message_keys)):
        required_params.append(par)
    else:
        optional_params.append(par)

In [29]:
print("Required params")
print(len(required_params))
required_params

Required params
8


['date',
 'date_unixtime',
 'from',
 'from_id',
 'id',
 'text',
 'text_entities',
 'type']

In [30]:
print("Required params")
print(len(optional_params))
optional_params

Required params
25


['author',
 'contact_information',
 'contact_vcard',
 'duration_seconds',
 'edited',
 'edited_unixtime',
 'file',
 'forwarded_from',
 'height',
 'inline_bot_buttons',
 'live_location_period_seconds',
 'location_information',
 'media_type',
 'mime_type',
 'performer',
 'photo',
 'poll',
 'reply_to_message_id',
 'reply_to_peer_id',
 'saved_from',
 'sticker_emoji',
 'thumbnail',
 'title',
 'via_bot',
 'width']

Таким образом мы получили 33 атрибута. 8 встречаются во всех *моих* сообщениях, 25 оставшихся - опциональны.

# Итоги
Приведенный выше анализ должен навести на мысль о возможности объектно-ориентированного подхода. Я бы предложил просто переложить структуру .json-файла на класс, но при этом не делать сложную структуру наследования, т.к. это выглядит излишним в следующей ситуации. Пусть у нас есть текстовое сообщение, к которому прикреплены 2 фотографии и 3 видео. В такой ситуации довольно затруднительно определить класс данного сообщения. Я предлагаю создать класс `Message` и хранить всю необходимую информацию о его типе и наличии вложений и т.п. в одноименных с .json-файлом полях.